In [1]:
pip install tensorflow-gpu

Note: you may need to restart the kernel to use updated packages.


**Mario game setup**

In [2]:
!pip install gym_super_mario_bros==7.3.0 nes_py


In [3]:

# Importing the game
import gym_super_mario_bros
# Importing the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Importing the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT


In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [5]:
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100000): 
    # Start the game to begin with 
    if done: 
        # Start the gamee
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
# Close the game
env.close()

C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


**Preprocessing the environment**

In [6]:
# Installing pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html


Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [7]:

# Installing the stable baselines for Reinforcement Learning stuff
!pip install stable-baselines3[extra]

In [8]:
# Importing the Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Importing the Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Importing the Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [9]:
# 1. Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale observation
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')

In [10]:

state = env.reset()

In [11]:
state, reward, done, info = env.step([5])

**Train the Reinforcement Learning model**

In [13]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [14]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [17]:
CHECKPOINT_DIRECTORY = './training/'
LOG_DIRECTORY = './logs/'

In [18]:
# Setup the model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIRECTORY)

In [19]:
# AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIRECTORY, learning_rate=0.000001, 
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [20]:
# Train the AI model, now AI model starts learning
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_1


C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 68  |
|    iterations      | 1   |
|    time_elapsed    | 7   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 78            |
|    iterations           | 2             |
|    time_elapsed         | 13            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 2.5036163e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.000593     |
|    learning_rate        | 1e-06         |
|    loss                 | 163           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000289     |
|    value_loss           | 430           |
-------------------------------------------
-----

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 13            |
|    time_elapsed         | 69            |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 1.3301615e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0132        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0674        |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000113     |
|    value_loss           | 0.743         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 14            |
|    time_elapsed         | 74  

-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 24            |
|    time_elapsed         | 127           |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 2.6861555e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | -0.00062      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.128         |
|    n_updates            | 230           |
|    policy_gradient_loss | -0.000427     |
|    value_loss           | 0.267         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 96            |
|    iterations           | 25            |
|    time_elapsed         | 132 

...

-------------------------------------------
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 1905          |
|    time_elapsed         | 12555         |
|    total_timesteps      | 975360        |
| train/                  |               |
|    approx_kl            | 0.00058011943 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.36         |
|    explained_variance   | 0.807         |
|    learning_rate        | 1e-06         |
|    loss                 | 311           |
|    n_updates            | 19040         |
|    policy_gradient_loss | -0.000643     |
|    value_loss           | 557           |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 1906          |
|    time_elapsed         | 1256

------------------------------------------
| time/                   |              |
|    fps                  | 77           |
|    iterations           | 1916         |
|    time_elapsed         | 12612        |
|    total_timesteps      | 980992       |
| train/                  |              |
|    approx_kl            | 0.0005057445 |
|    clip_fraction        | 0.000195     |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.374       |
|    explained_variance   | 0.844        |
|    learning_rate        | 1e-06        |
|    loss                 | 121          |
|    n_updates            | 19150        |
|    policy_gradient_loss | -8.41e-05    |
|    value_loss           | 419          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 77           |
|    iterations           | 1917         |
|    time_elapsed         | 12617        |
|    total_

-------------------------------------------
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 1927          |
|    time_elapsed         | 12667         |
|    total_timesteps      | 986624        |
| train/                  |               |
|    approx_kl            | 0.00035780668 |
|    clip_fraction        | 0.00117       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.443        |
|    explained_variance   | 0.873         |
|    learning_rate        | 1e-06         |
|    loss                 | 79.1          |
|    n_updates            | 19260         |
|    policy_gradient_loss | -0.00011      |
|    value_loss           | 213           |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 77           |
|    iterations           | 1928         |
|    time_elapsed         | 12672   

-------------------------------------------
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 1938          |
|    time_elapsed         | 12723         |
|    total_timesteps      | 992256        |
| train/                  |               |
|    approx_kl            | 0.00070843333 |
|    clip_fraction        | 0.00195       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.514        |
|    explained_variance   | 0.767         |
|    learning_rate        | 1e-06         |
|    loss                 | 161           |
|    n_updates            | 19370         |
|    policy_gradient_loss | -0.000687     |
|    value_loss           | 406           |
-------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 77          |
|    iterations           | 1939        |
|    time_elapsed         | 12728       

------------------------------------------
| time/                   |              |
|    fps                  | 78           |
|    iterations           | 1949         |
|    time_elapsed         | 12779        |
|    total_timesteps      | 997888       |
| train/                  |              |
|    approx_kl            | 0.0019701899 |
|    clip_fraction        | 0.0283       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.47        |
|    explained_variance   | 0.812        |
|    learning_rate        | 1e-06        |
|    loss                 | 53.6         |
|    n_updates            | 19480        |
|    policy_gradient_loss | -0.00317     |
|    value_loss           | 118          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 78           |
|    iterations           | 1950         |
|    time_elapsed         | 12784        |
|    total_

In [22]:
model.save('Testmodel')

**Testing and Loading the Model**

In [23]:
# Loading the model
model = PPO.load('./training/best_model_1000000')

In [24]:
state = env.reset()

In [ ]:
# Starting the game 
state = env.reset()
# Looping through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()